In [1]:
from __future__ import division
from sklearn.cluster import  MiniBatchKMeans, KMeans, DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd
import math
from pylab import *
from scipy.spatial.distance import cdist
from scipy import spatial
from mpl_toolkits.mplot3d import Axes3D

In [2]:
from Parameters import *
from logistics_functions import *
from Parameters import time_between_catridge_regeneration, collection_times_per_year

In [3]:
n_regen = 30
n_collection = 1
schedule = 'scheduled'
path = '..\shapefiles\\building_block_people_WGS_2.csv'
nx = int(sqrt(30))
ny = int(ceil(n_regen/nx))

In [4]:
bounding_box = [37.729445, 37.797265, -122.387494, -122.500237]
Xgrid = make_grid(bounding_box, nx, ny)
grid_coord=[]

for row in Xgrid:
    xgrid=row[0]
    ygrid=row[1]
    grid_coords = (xgrid, ygrid)
    grid_coord.append(grid_coords)

grid_coord_df=pd.DataFrame(grid_coord, columns=['lat', 'lon'])

In [5]:
building_virtual_buildings_df = get_virtual_buildings(path)
building_SF_points=building_virtual_buildings_df[['lat_lat','lon_lon']]

In [6]:
tree = spatial.KDTree(list(zip(grid_coord_df['lat'], grid_coord_df['lon'])))

In [7]:
building_SF_points_array = list(zip(building_SF_points['lat_lat'],building_SF_points['lon_lon']))

In [8]:
k_means_labels_regen = tree.query(building_SF_points_array)[1]

In [9]:
k_means_labels_collection, k_means_cluster_centers_collection = clustering(grid_coord, n_collection)

In [10]:
def logistics_distances(building_virtual_buildings_df, k_means_labels_regen, k_means_cluster_centers_regen, schedule):
    if schedule == 'scheduled':
        distance_regeneration = find_distance_regeneration_scheduled(building_virtual_buildings_df, k_means_labels_regen)
        distance_regeneration['total_dist_m'] = distance_regeneration['total_dist_m']*365/time_between_catridge_regeneration

    elif schedule == 'unscheduled':
        distance_regeneration = find_distance_regeneration_scheduled(building_virtual_buildings_df, k_means_labels_regen)
        distance_regeneration['total_dist_m'] = distance_regeneration['total_dist_m']*365

    else:
        print ("Error scheduling was not specified")
        return

    distance_collection = find_distance_collection(k_means_cluster_centers_regen)*collection_times_per_year
    return distance_regeneration, distance_collection

In [11]:
distance_regeneration, distance_collection = logistics_distances(building_virtual_buildings_df, k_means_labels_regen, grid_coord, schedule)

In [12]:
distance_regeneration

,cluster,num_people,trucks_num,total_dist_m
0,0,2.0,0.01,0.000000
1,1,19736.0,9.62,3199.345434
2,2,17393.0,9.30,2654.500819
3,3,23772.0,14.40,3881.245183
4,4,6365.0,3.54,1058.701992
5,5,59.0,0.08,27.953131
6,6,34282.0,13.71,4608.790468
7,7,21328.0,10.43,3442.702632
8,8,24768.0,15.56,5185.488502
9,9,23644.0,9.51,3825.211279
